In [1]:
# from yandex_chain import YandexEmbeddings
from langchain_community.vectorstores import FAISS

import os

In [ ]:
!pip install accelerate

In [26]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = 'IlyaGusev/saiga_llama3_8b'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    # load_in_8bit=True,
    torch_dtype = torch.bfloat16,
    device_map = "auto"
)



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
model.eval()

In [51]:
from transformers import GenerationConfig
generation_config = GenerationConfig.from_pretrained(model_id)
print(generation_config)

GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": 128009,
  "max_new_tokens": 1536,
  "pad_token_id": 128000,
  "repetition_penalty": 1.12,
  "temperature": 0.2,
  "top_k": 30,
  "top_p": 0.9
}



In [52]:
generation_config.max_new_tokens = 512
generation_config.temperature = 0.2
generation_config.top_k= 80

In [37]:
def get_llm_answer(query, chunks_join):
    user_prompt = '''Используй только следующий контекст, чтобы очень кратко ответить на вопрос в конце.
    Не пытайся выдумывать ответ.
    Контекст:
    ===========
    {chunks_join}
    ===========
    Вопрос:
    ===========
    {query}'''.format(chunks_join=chunks_join, query=query)
    
    SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."
    RESPONSE_TEMPLATE = "<|im_start|>assistant\n"
    
    prompt = f'''<|im_start|>system\n{SYSTEM_PROMPT}<|im_end|>\n<|im_start|>user\n{user_prompt}<|im_end|>\n{RESPONSE_TEMPLATE}'''
    
    def generate(model, tokenizer, prompt):
        data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
        data = {k: v.to(model.device) for k, v in data.items()}
        output_ids = model.generate(
            **data,
            generation_config=generation_config
        )[0]
        output_ids = output_ids[len(data["input_ids"][0]) :]
        output = tokenizer.decode(output_ids, skip_special_tokens=True)
        return output.strip()
    
    response = generate(model, tokenizer, prompt)
    
    return response

### Поиск чанков с близкими эмбеддингами из векторной базы

In [6]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

In [7]:
embeddings = HuggingFaceEmbeddings(model_name="cointegrated/LaBSE-en-ru")

/home/yagor/anaconda3/envs/torch/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
# /home/yagor/Рабочий стол/atomic/AtomHack/data/unstructured_vdb
db2 = FAISS.load_local(
    os.path.join("..", "data", 'unstructured_vdb'),
    embeddings,
    allow_dangerous_deserialization = True
)

In [15]:
query = "Что является целью интеграционного сценария «Массовая выгрузка документов из ЕОСДО?"
docs = db2.similarity_search(query)

In [16]:
docs

[Document(page_content='Автор АО «Гринатом» Инструкция пользователей «Интеграция 1С ERP 2.0: \nЦифровой Росатом с ЕОСДО » Страница 10 \nСтраниц 35 \nВерсия  6 \n1.2 Интеграционный сценарий «Массовая выгрузка доку ментов из \nЕОСДО» \nСхематично интеграционный сценарий изображён на Рис унке 1.2: \nРисунок 1.2. Сценарий «Массовая выгрузка документов  из ЕОСДО» \nФормирование \nреестра и отправка \nреестра в ЕОСДО \nПолучение и \nобработка реестра Выгрузка скан-\nобразов \nдокументов на \nфайловый сервер Подтверждение \nзавершения \nвыгрузки \nM .O.3 M .I.3\n \nЦелью интеграционного сценария «Массовая выгрузка д окументов из \nЕОСДО» является автоматизация предоставления реестр а скан-образов \nдокументов в контролирующие органы. \nЗависимые бизнес-процессы:  \n\uf0b7 D-1C1-1.10.19 Формирование налоговой декларации по НП; \n\uf0b7 D-1C1-1.10.15 Формирование налоговой декларации по НДС; \n\uf0b7 D-1C1-1.10.35 Учет средств целевого финансирования;  \n\uf0b7 D-1C1-1.10.40 Формирование бухгал

In [27]:
print(docs[0].page_content)
page = docs[0].metadata["source"].split("/")[-1].split("_")[-1].split(".")[0]
file_name = docs[0].metadata['source'].split('/')[-2] + ".pdf"
print(f"Страница: {page}", f"Исходный документ: {file_name}")

Автор АО «Гринатом» Инструкция пользователей «Интеграция 1С ERP 2.0: 
Цифровой Росатом с ЕОСДО » Страница 10 
Страниц 35 
Версия  6 
1.2 Интеграционный сценарий «Массовая выгрузка доку ментов из 
ЕОСДО» 
Схематично интеграционный сценарий изображён на Рис унке 1.2: 
Рисунок 1.2. Сценарий «Массовая выгрузка документов  из ЕОСДО» 
Формирование 
реестра и отправка 
реестра в ЕОСДО 
Получение и 
обработка реестра Выгрузка скан-
образов 
документов на 
файловый сервер Подтверждение 
завершения 
выгрузки 
M .O.3 M .I.3
 
Целью интеграционного сценария «Массовая выгрузка д окументов из 
ЕОСДО» является автоматизация предоставления реестр а скан-образов 
документов в контролирующие органы. 
Зависимые бизнес-процессы:  
 D-1C1-1.10.19 Формирование налоговой декларации по НП; 
 D-1C1-1.10.15 Формирование налоговой декларации по НДС; 
 D-1C1-1.10.35 Учет средств целевого финансирования;  
 D-1C1-1.10.40 Формирование бухгалтерской отчетности .
Страница: 10 Исходный документ: Инструкция Интегра

In [ ]:
# !pip install torch==2.2.2

In [40]:
result = get_llm_answer(query, docs[0].page_content)

In [41]:
result

'Целью интеграционного сценария «Массовая выгрузка документов из ЕОСДО» является автоматизация предоставления реестра скан-образов документов в контролирующие органы. <|im_end|>'

In [48]:
query = """Посмотри счета учета расчетов, настраиваемые с использованием 
элементов списка. Выведи все в виде групп в соответствии с их применением в операциях"""
docs = db2.similarity_search(query)

In [49]:
docs

[Document(page_content='отражаем в регламентированном учете. Суммы пересчитываются исходя из', metadata={'source': '/home/yagor/Рабочий стол/atomic/AtomHack/data/txt_data/Инструкция_D_1C1_1_10_04_Учет_банковских_операций_НФ_v2_28_1/page_110.txt'}),
 Document(page_content='при настройке видов сертификатов.  \n\uf0b7 Счета учета расчет ов с контрагентами, настраиваемые с использованием \nэлементов списка  Группы настроек финансового учета расчетов, представлены в \nвиде групп в соответствии с их применением в операциях:  \no Расчетов с клиентами;  \no Расчетов с поставщиками;  \no Расчетов с комиссионерами;  \no Расчетов с комитентами;  \no Расчетов с кредиторами;  \no Расчетов с дебиторами;  \no Расчетов с лизингодателями.  \n\uf0b7 Производства  – задаются счета учета прямых производственных затрат \n(номенклатурные затраты на счетах 20, 23, 29) для подразделений в рамках \nкаждой организац ии. \n\uf0b7 Внеоборотных активов  – счета учета внеоборотных активов, настраиваемые с \nиспольз

In [50]:
# большая температура -> придуманный ответ
result = get_llm_answer(query, docs[0].page_content)
print(result)

Вывод всех счетов учета расчетов, настраиваемых с использованием элементов списка, в виде групп по их применению в операциях: 
1. **Группа 1: Расходы на материалы**
   - Счет 101: Материалы для производства
   - Счет 102: Инструменты и оборудование

2. **Группа 2: Зарплата сотрудников**
   - Счет 201: Заработная плата
   - Счет 202: Начисления на социальное страхование

3. **Группа 3: Аренда и коммунальные платежи**
   - Счет 301: Аренда помещений
   - Счет 302: Электроэнергия
   - Счет 303: Вода

4. **Группа 4: Транспортные расходы**
   - Счет 401: Грузоперевозки
   - Счет 402: Автотранспорт

5. **Группа 5: Общепроизводственные расходы**
   - Счет 501: Услуги сторонних организаций
   - Счет 502: Ремонт и техническое обслуживание оборудования

6. **Группа 6: Прочие расходы**
   - Счет 601: Коммуникационные услуги
   - Счет 602: Печати и бумаги

7. **Группа 7: Прибыль**
   - Счет 701: Продажа товаров (услуг)
   - Счет 702: Отчисление прибыли

8. **Группа 8: Налоговые обязательства**
   

In [53]:
result = get_llm_answer(query, docs[0].page_content)
print(result)